# Exercise 2: Temporal Hello World — Solution

Complete implementation of the Temporal Hello World workflow with an activity.

## Prerequisites

Make sure the Temporal dev server is running before executing the solution.

In [10]:
%pip install --quiet temporalio rich

import asyncio
from datetime import timedelta
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker

console = Console()



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Run the Solution

Use the cell below to run the fully implemented solution script.

In [11]:
# Execute the solution workflow
async def run_solution() -> None:
    console.print("\n[bold cyan]🚀 Exercise 2: Temporal Hello World — Solution[/bold cyan]\n")
    client = await Client.connect("localhost:7233")
    task_queue = "hello-world-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[HelloWorkflow],
        activities=[process_data],
    ):
        workflow_id = "hello-workflow-1"
        result = await client.execute_workflow(
            HelloWorkflow.run,
            "Temporal",
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]✅ Workflow Result:[/bold green] {result}\n")
    console.print(
        "[yellow]View execution history:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}\n"
    )

asyncio.run(run_solution())


RuntimeError: asyncio.run() cannot be called from a running event loop

/home/vscode/.local/lib/python3.11/site-packages/pygments/regexopt.py:39: RuntimeWarning: coroutine 'run_solution' was never awaited
  return open_paren + regex_opt_inner(strings[1:], '(?:') \


## Source Code

Review the solution implementation across the key modules.

In [8]:
@activity.defn
async def process_data(data: str) -> str:
    """Activity that simulates work by upper-casing the payload."""
    activity.logger.info("🔧 Activity started")
    await asyncio.sleep(1)
    result = f"Processed: {data.upper()}"
    activity.logger.info("✅ Activity completed")
    return result

@workflow.defn
class HelloWorkflow:
    """Workflow that orchestrates the activity call."""

    @workflow.run
    async def run(self, name: str) -> str:
        workflow.logger.info("🚀 Workflow started")
        result = await workflow.execute_activity(
            process_data,
            args=[f"Hello {name}"],
            start_to_close_timeout=timedelta(seconds=10),
        )
        workflow.logger.info("✅ Workflow finished")
        return f"Workflow result: {result}"
